
# Eliminação de Gauss 



## Estrutura

1. **Nome do método:** Eliminação de Gauss (com pivoteamento parcial opcional).  
2. **Ideia central:** Transformar o sistema linear $A \mathbf{x} = \mathbf{b}$ em um sistema triangular superior $U \mathbf{x} = \mathbf{c}$ por meio de **operações elementares de linha** (zerando coeficientes abaixo da diagonal). Em seguida, resolver por **retrossubstituição**.  
3. **Entrada/Saída:**  
   - **Entrada:** matriz quadrada $A \in \mathbb{R}^{n \times n}$ e vetor coluna $b \in \mathbb{R}^{n}$.  
   - **Saída:** solução $\mathbf{x}$ (se existir/for única); opcionalmente as matrizes intermediárias e métricas de erro.



## Código — Usando bibliotecas (NumPy / SciPy)

A forma mais prática em Python é usar `numpy.linalg.solve` (ou `scipy.linalg.solve`).  
Internamente, essas rotinas usam fatorações numéricas estáveis (geralmente **LU com pivoteamento parcial**), que são computacionalmente equivalentes à eliminação de Gauss, porém mais robustas.


In [ ]:
import numpy as np   # importa a biblioteca NumPy, que ajuda a trabalhar com matrizes e cálculos numéricos

# Criamos a matriz A (3x3), que representa os coeficientes do sistema linear
A = np.array([[2.0, 1.0, -1.0],
              [-3.0, -1.0,  2.0],
              [-2.0, 1.0,  2.0]], dtype=float)

# Criamos o vetor b, que representa o lado direito do sistema Ax = b
b = np.array([8.0, -11.0, -3.0], dtype=float)

# Usamos o NumPy para resolver o sistema de equações lineares Ax = b
x_np = np.linalg.solve(A, b)

# Calculamos o "resíduo", que é a diferença entre Ax e b, ou seja, o erro da solução
residuo = np.linalg.norm(A @ x_np - b)

# Mostramos a solução encontrada para o vetor x
print("Solução com NumPy:", x_np)

# Mostramos o tamanho do resíduo (‖Ax - b‖2), que deve ser bem próximo de 0 se a solução estiver correta
print("‖Ax - b‖2 =", residuo)



## Código — Implementação do método (do zero) com **pivoteamento parcial**

A seguir, implementamos a **Eliminação de Gauss** manualmente. O pivoteamento parcial troca a linha corrente pela que possui o **maior valor absoluto** no pivô da coluna, reduzindo instabilidades numéricas quando o pivô é pequeno.


In [ ]:
import numpy as np

def gauss_elimination(A, b, pivot=True, return_U=True):
    """
    Realiza eliminação de Gauss com pivoteamento parcial opcional.

    Parâmetros
    ----------
    A : array_like (n, n)   -> matriz dos coeficientes
    b : array_like (n,)     -> vetor do lado direito
    pivot : bool            -> se True, ativa o pivoteamento parcial
    return_U : bool         -> se True, retorna também a matriz U (triangular superior) e o b modificado
    """

    # Converte A e b em arrays do NumPy (tipo float, para evitar erro de inteiros)
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]   # número de linhas/colunas da matriz

    # -------------------------------
    # 1ª Etapa: Eliminação progressiva (forward elimination)
    # -------------------------------
    for k in range(n-1):   # percorre cada coluna (exceto a última)
        
        # Pivoteamento parcial (troca de linhas para evitar divisão por 0 ou instabilidade numérica)
        if pivot:
            i_max = k + np.argmax(np.abs(A[k:, k]))   # índice do maior elemento da coluna abaixo de k
            if A[i_max, k] == 0.0:   # se o maior for zero, não há solução única
                raise np.linalg.LinAlgError("Matriz singular detectada durante o pivoteamento.")
            if i_max != k:   # troca de linha
                A[[k, i_max], :] = A[[i_max, k], :]
                b[[k, i_max]] = b[[i_max, k]]
        else:   # caso não use pivoteamento
            if A[k, k] == 0.0:
                raise np.linalg.LinAlgError("Pivô zero sem pivoteamento — método falha.")

        # Eliminação dos elementos abaixo do pivô
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]        # multiplicador
            A[i, k:] -= m * A[k, k:]     # zera o elemento A[i,k] e ajusta o resto da linha
            b[i] -= m * b[k]             # ajusta também o vetor b

    # -------------------------------
    # Verificação de singularidade
    # -------------------------------
    if np.isclose(A[-1, -1], 0.0):
        raise np.linalg.LinAlgError("Matriz singular ou quase singular; não há solução única.")

    # -------------------------------
    # 2ª Etapa: Retrossubstituição (back substitution)
    # -------------------------------
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):   # começa da última linha até a primeira
        s = np.dot(A[i, i+1:], x[i+1:])   # soma dos termos já conhecidos
        x[i] = (b[i] - s) / A[i, i]       # resolve a incógnita

    # Se return_U=True, retorna também a matriz triangular superior U e o b modificado
    if return_U:
        return x, A, b
    return x


# -------------------------------
# Teste da função
# -------------------------------
A_test = np.array([[2.0, 1.0, -1.0],
                   [-3.0, -1.0, 2.0],
                   [-2.0, 1.0, 2.0]], dtype=float)
b_test = np.array([8.0, -11.0, -3.0], dtype=float)

# Resolve manualmente com Gauss
x_manual, U, b_mod = gauss_elimination(A_test, b_test, pivot=True, return_U=True)

print("Solução (manual):", x_manual)   # vetor solução
print("U (triangular superior):\n", U) # matriz já triangularizada
print("b modificado:\n", b_mod)        # vetor b após as operações
print("‖Ax - b‖2 =", np.linalg.norm(A_test @ x_manual - b_test))  # verifica o resíduo


## Características

1. **Tipo de método: direto** (não iterativo). A solução é obtida após um número finito de operações aritméticas (assumindo aritmética exata).  

2. **Requisitos/Condições de uso:**  
   - $A$ deve ser **quadrada** e **não singular** (determinante diferente de zero).  
   - Sem pivoteamento, o método pode falhar com **pivôs nulos** ou **quase nulos**; com **pivoteamento parcial** a robustez aumenta bastante.  
   - Não exige que $A$ seja simétrica ou definida positiva (essas propriedades são relevantes para métodos específicos como **Cholesky**, que requer $A$ simétrica definida positiva).  

3. **Complexidade:** custo assintótico $ \mathcal{O}(n^3) $ operações; memória $ \mathcal{O}(n^2) $.


## Possíveis problemas

1. **Quando falha?**  
   - **Matriz singular**: não existe solução única; durante a eliminação surge pivô zero.  
   - **Pivôs muito pequenos** (sem pivoteamento): levam a **erros de arredondamento** e instabilidade numérica.  
   - **Sistemas mal-condicionados**: pequena mudança em $b$ pode causar grande variação em $\mathbf{x}$; o método entrega uma solução, mas a **confiança** deve ser avaliada via número de condição $\text{cond}(A)$.  

2. **É considerado eficiente?**  
   - Para **tamanhos moderados** de $n$ (até algumas dezenas de milhares, dependendo do hardware/uso de BLAS), sim — pois é $ \mathcal{O}(n^3) $ e com boa implementação vetorizada.  
   - Para matrizes **esparsas** muito grandes, variantes especializadas (eliminação esparsa, solvers iterativos) podem ser superiores.


## Qualidades

1. **Vantagens:**  
   - Método **determinístico** e **direto**, com solução em passos finitos; base das rotinas `solve`/LU.  
   - Com **pivoteamento parcial**, é **estável** para uma ampla classe de problemas.  
   - Serve como **base conceitual** para outras fatorações (LU, Cholesky) e para análise de complexidade.  

2. **Quando é a melhor escolha?**  
   - Sistemas **densos** de tamanho pequeno a médio.  
   - Quando se precisa de **boa robustez** e **reprodutibilidade**.  
   - Quando o mesmo $A$ será reutilizado com vários $b$: neste caso, usa-se a **fatoração LU** uma vez e resolve-se muitas vezes com custo $ \mathcal{O}(n^2) $ por novo vetor $b$.



## Experimentos rápidos de validação

Abaixo comparamos a solução manual com `numpy.linalg.solve`, avaliando resíduo e tempo em problemas aleatórios.


In [ ]:
import numpy as np     # biblioteca para cálculos com matrizes
import time            # medir tempo de execução

# Função que resolve Ax = b com NumPy e com método manual
# e compara erro (||Ax-b||) e tempo
def solve_and_metrics(A, b):
    # --- Usando NumPy ---
    t0 = time.time()                  
    x_np = np.linalg.solve(A, b)      # resolve sistema com NumPy
    t1 = time.time()                  
    r_np = np.linalg.norm(A @ x_np - b)   # erro da solução NumPy

    # --- Usando eliminação de Gauss manual ---
    x_man = gauss_elimination(A, b, pivot=True, return_U=False)
    t2 = time.time()
    r_man = np.linalg.norm(A @ x_man - b) # erro da solução manual

    # retorna dicionário com resultados
    return {"||Ax-b|| (NumPy)": r_np, "tempo NumPy (s)": t1 - t0,
            "||Ax-b|| (Manual)": r_man, "tempo Manual (s)": t2 - t1}

# --- Criando dados de teste ---
np.random.seed(0)        # fixa aleatoriedade
n = 6                    # tamanho da matriz
A = np.random.randn(n, n)  # matriz aleatória

# garante que A não seja quase singular (det ~ 0)
while abs(np.linalg.det(A)) < 1e-6:
    A = np.random.randn(n, n)

b = np.random.randn(n)   # vetor de termos independentes

# --- Chamando a função ---
metrics = solve_and_metrics(A, b)   # calcula métricas
metrics                             # mostra os resultados


## Diagnóstico: número de condição

O **número de condição** (cond) de \(A\) dá uma ideia de sensibilidade do sistema. Quanto maior, mais **mal condicionado** e menos confiável será qualquer método direto em aritmética de ponto flutuante.


In [ ]:
import numpy as np   # importa a biblioteca NumPy, útil para cálculos numéricos e matrizes

# Função que analisa o condicionamento de uma matriz
def diagnostico_condicionamento(A):
    # Calcula o número de condição da matriz na norma 2 (baseado em autovalores singulares)
    cond2 = np.linalg.cond(A, p=2)

    # Exibe o número de condição em notação científica (ex.: 1.23e+04)
    print(f"Número de condição (2-norma): {cond2: .2e}")

    # Classifica o condicionamento da matriz com base no valor calculado
    if cond2 < 1e3:
        print("→ Bem condicionado para a maioria dos fins práticos.")
    elif cond2 < 1e8:
        print("→ Condicionamento moderado: atenção ao arredondamento.")
    else:
        print("→ Muito mal condicionado: resultados podem perder muitos dígitos de precisão.")

# Matriz quase singular (determinante próximo de zero, tende a ser mal condicionada)
A1 = np.array([[1, 1], [1, 1.0001]], dtype=float)

# Matriz identidade 4x4 (sempre bem condicionada, número de condição = 1)
A2 = np.eye(4)

# Testa a matriz A1
print("A1:")
diagnostico_condicionamento(A1)

# Testa a matriz A2
print("\nA2:")
diagnostico_condicionamento(A2)


## Conclusão

- A **Eliminação de Gauss** é um método **direto e fundamental** para resolver sistemas lineares.  
- Em prática, usa-se `numpy.linalg.solve`/`scipy.linalg.solve`, que implementam rotinas **otimizadas e estáveis (LU com pivoteamento)**.  
- A **versão manual com pivoteamento parcial** reproduz a ideia do método e obtém bons resultados em aritmética de ponto flutuante.  
- Avaliar **resíduos** e **condicionamento** ajuda a julgar a confiabilidade das soluções.

> **Observação para o relatório:** edite este notebook e inclua seu nome, turma e comentários/conclusões pessoais onde achar relevante.


## Referências

1. **Gilbert Strang** — *Linear Algebra and Its Applications*, 4ª edição, Brooks Cole, 2005.  
2. **David C. Lay** — *Álgebra Linear e Suas Aplicações*, 4ª edição, Bookman, 2012.  
3. **Paulo Winterle** — *Vetores e Geometria Analítica*.  
4. **Gene H. Golub & Charles F. Van Loan** — *Matrix Computations*, 4ª edição, Johns Hopkins University Press, 2013.  
5. **NumPy Documentation** — [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html)  
6. **SciPy Documentation** — [`scipy.linalg.solve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html)  
7. **Wikipedia** — [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination)  
